# This is a scratch space for building functions
After functions are made, move them to the corresponding folder. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread

In [ ]:
a = imread("../images/B005_1.tif")
b = imread("../images/B005_2.tif")

fig, axs = plt.subplots(1, 2, figsize=(9, 4))
axs[0].imshow(a, cmap=plt.cm.gray)
axs[1].imshow(b, cmap=plt.cm.gray)
plt.show()